Reading the Feature engineered csv

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Load the data
df = pd.read_csv('lung_cancer_mortality_featured.csv')

In [20]:
# Display basic information about the dataset
print("Basic Info:")
df.info()

Basic Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700000 entries, 0 to 699999
Columns: 225 entries, age to survived
dtypes: bool(49), float64(161), int64(15)
memory usage: 972.7 MB


In [21]:
# Checking missing values
print("\nMissing Values per Column:")
print(df.isna().sum().sort_values(ascending=False))


Missing Values per Column:
survived / other_cancer             498133
other_cancer / survived             498133
other_cancer / cirrhosis            495881
cirrhosis / other_cancer            495881
cirrhosis / survived                422587
                                     ...  
cholesterol_level + hypertension         0
cholesterol_level + other_cancer         0
cholesterol_level + survived             0
cirrhosis + hypertension                 0
survived                                 0
Length: 225, dtype: int64


In [22]:
# Display summary statistics for numeric columns
print("\nSummary Statistics:")
print(df.describe())


Summary Statistics:


/Users/shubhamlaxmikantdeshmukh/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/Users/shubhamlaxmikantdeshmukh/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/Users/shubhamlaxmikantdeshmukh/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/Users/shubhamlaxmikantdeshmukh/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/Users/shubhamlaxmikantdeshmukh/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/Users/shubhamlaxmikantdeshmukh/anaconda3/lib/python3.11/site-packages

                 age            bmi  cholesterol_level   hypertension  \
count  700000.000000  700000.000000      700000.000000  700000.000000   
mean       55.001127      30.488436         233.641653       0.750137   
std         9.996749       8.370034          43.439823       0.432934   
min         4.000000      16.000000         150.000000       0.000000   
25%        48.000000      23.200000         196.000000       1.000000   
50%        55.000000      30.500000         242.000000       1.000000   
75%        62.000000      37.700000         271.000000       1.000000   
max       104.000000      45.000000         300.000000       1.000000   

              asthma      cirrhosis   other_cancer  ABSOLUTE(age)  \
count  700000.000000  700000.000000  700000.000000  700000.000000   
mean        0.469134       0.226369       0.087806      55.001127   
std         0.499047       0.418481       0.283012       9.996749   
min         0.000000       0.000000       0.000000       4.000000 

In [23]:
# Display the number of unique values in each column
print("\nUnique Value Count per Column:")
print(df.nunique())


Unique Value Count per Column:
age                                    94
bmi                                   291
cholesterol_level                     151
hypertension                            2
asthma                                  2
                                     ... 
TIME_SINCE(end_treatment_date)       4192
YEAR(beginning_of_treatment_date)      11
YEAR(diagnosis_date)                   11
YEAR(end_treatment_date)               13
survived                                2
Length: 225, dtype: int64


In [24]:
# Checking for infinite values
print("\nInfinite Value Count per Column:")
infinite_counts = df.apply(lambda col: (col == float('inf')).sum() + (col == float('-inf')).sum())
print(infinite_counts[infinite_counts > 0])


Infinite Value Count per Column:
age / asthma                        371606
age / cirrhosis                     541542
age / hypertension                  174904
age / other_cancer                  638536
age / survived                      546178
asthma / cirrhosis                  246615
asthma / hypertension                65728
asthma / other_cancer               295647
asthma / survived                   256122
bmi / asthma                        371606
bmi / cirrhosis                     541542
bmi / hypertension                  174904
bmi / other_cancer                  638536
bmi / survived                      546178
cholesterol_level / asthma          371606
cholesterol_level / cirrhosis       541542
cholesterol_level / hypertension    174904
cholesterol_level / other_cancer    638536
cholesterol_level / survived        546178
cirrhosis / asthma                   76679
cirrhosis / hypertension             27280
cirrhosis / other_cancer            142655
cirrhosis / survived

In [26]:
import numpy as np

# Identify columns with positive or negative infinite values
cols_with_infinite = df.columns[(df == float('inf')).any() | (df == float('-inf')).any()]

# Drop columns with infinite values
df_cleaned = df.drop(columns=cols_with_infinite)

# Print columns that were removed
print("Columns with infinite values removed:")
print(cols_with_infinite)


Columns with infinite values removed:
Index(['age / asthma', 'age / cirrhosis', 'age / hypertension',
       'age / other_cancer', 'age / survived', 'asthma / cirrhosis',
       'asthma / hypertension', 'asthma / other_cancer', 'asthma / survived',
       'bmi / asthma', 'bmi / cirrhosis', 'bmi / hypertension',
       'bmi / other_cancer', 'bmi / survived', 'cholesterol_level / asthma',
       'cholesterol_level / cirrhosis', 'cholesterol_level / hypertension',
       'cholesterol_level / other_cancer', 'cholesterol_level / survived',
       'cirrhosis / asthma', 'cirrhosis / hypertension',
       'cirrhosis / other_cancer', 'cirrhosis / survived',
       'hypertension / asthma', 'hypertension / cirrhosis',
       'hypertension / other_cancer', 'hypertension / survived',
       'other_cancer / asthma', 'other_cancer / cirrhosis',
       'other_cancer / hypertension', 'other_cancer / survived',
       'survived / asthma', 'survived / cirrhosis', 'survived / hypertension',
       'surviv

In [27]:
df_cleaned.describe()

,age,bmi,cholesterol_level,hypertension,asthma,cirrhosis,other_cancer,ABSOLUTE(age),ABSOLUTE(asthma),ABSOLUTE(bmi),...,hypertension - other_cancer,hypertension - survived,other_cancer - survived,TIME_SINCE(beginning_of_treatment_date),TIME_SINCE(diagnosis_date),TIME_SINCE(end_treatment_date),YEAR(beginning_of_treatment_date),YEAR(diagnosis_date),YEAR(end_treatment_date),survived
count,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,...,700000.000000,700000.000000,700000.00000,7.000000e+05,7.000000e+05,7.000000e+05,700000.000000,700000.000000,700000.000000,700000.000000
mean,55.001127,30.488436,233.641653,0.750137,0.469134,0.226369,0.087806,55.001127,0.469134,30.488436,...,0.662331,0.530391,-0.13194,1.680554e+08,1.688751e+08,1.292846e+08,2018.948910,2018.922931,2020.177826,0.219746
std,9.996749,8.370034,43.439823,0.432934,0.499047,0.418481,0.283012,9.996749,0.499047,8.370034,...,0.499892,0.598913,0.50180,9.107841e+07,9.107752e+07,9.188628e+07,2.914471,2.913788,2.925946,0.414075
min,4.000000,16.000000,150.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,16.000000,...,-1.000000,-1.000000,-1.00000,8.705668e+06,1.129767e+07,-5.168793e+07,2014.000000,2014.000000,2014.000000,0.000000
25%,48.000000,23.200000,196.000000,1.000000,0.000000,0.000000,0.000000,48.000000,0.000000,23.200000,...,0.000000,0.000000,0.00000,8.905767e+07,8.992167e+07,5.043687e+07,2016.000000,2016.000000,2018.000000,0.000000
50%,55.000000,30.500000,242.000000,1.000000,0.000000,0.000000,0.000000,55.000000,0.000000,30.500000,...,1.000000,1.000000,0.00000,1.679409e+08,1.687185e+08,1.290609e+08,2019.000000,2019.000000,2020.000000,0.000000
75%,62.000000,37.700000,271.000000,1.000000,1.000000,0.000000,0.000000,62.000000,1.000000,37.700000,...,1.000000,1.000000,0.00000,2.469969e+08,2.477745e+08,2.081169e+08,2021.000000,2021.000000,2023.000000,0.000000
max,104.000000,45.000000,300.000000,1.000000,1.000000,1.000000,1.000000,104.000000,1.000000,45.000000,...,1.000000,1.000000,1.00000,3.265713e+08,3.266577e+08,3.107601e+08,2024.000000,2024.000000,2026.000000,1.000000


In [30]:
# Checking missing values
print("\nMissing Values per Column:")
print(df_cleaned.isna().sum().sort_values(ascending=False))
print("Total missing values in the dataframe: {}".format(df_cleaned.isna().sum().sum()))



Missing Values per Column:
age                           0
asthma * other_cancer         0
age * cholesterol_level       0
age * cirrhosis               0
age * hypertension            0
                             ..
asthma + bmi                  0
asthma + cholesterol_level    0
asthma + cirrhosis            0
asthma + hypertension         0
survived                      0
Length: 190, dtype: int64
Total missing values in the dataframe: 0


In [31]:
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

# Separate features (X) and target (y)
X = df_cleaned.drop(columns=['survived'])
y = df_cleaned['survived']

# Drop rows with missing values
X_cleaned = X.dropna()
y_cleaned = y[X_cleaned.index]  # Keep target variable aligned

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_cleaned, y_cleaned, test_size=0.3, random_state=42)

# Initialize Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model
rf.fit(X_train, y_train)

# Get feature importances
feature_importances = pd.DataFrame({'feature': X_train.columns, 'importance': rf.feature_importances_})

# Sort by importance
feature_importances = feature_importances.sort_values(by='importance', ascending=False)

# Display the most important features
print("Top 10 Important Features:")
print(feature_importances.head(10))


Top 10 Important Features:
                          feature  importance
136                bmi * survived    0.113532
107                survived / age    0.105143
154          PERCENTILE(survived)    0.103077
108                survived / bmi    0.082078
125                age * survived    0.079237
57             ABSOLUTE(survived)    0.076429
109  survived / cholesterol_level    0.072805
182       other_cancer - survived    0.061466
145       hypertension * survived    0.048821
140  cholesterol_level * survived    0.047553


In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split

# Separate features (X) and target (y)
X = df_cleaned.drop(columns=['survived'])
y = df_cleaned['survived']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize Logistic Regression model
logreg = LogisticRegression(max_iter=1000)

# Use RFE to select top features
rfe = RFE(logreg, n_features_to_select=10)  # You can change the number of features here
rfe.fit(X_train, y_train)

# Get selected features
selected_features = X_train.columns[rfe.support_]

# Display selected features
print("Top Selected Features by RFE:")
print(selected_features)


Top Selected Features by RFE:
Index(['age + cholesterol_level', 'age * bmi', 'age * cholesterol_level',
       'bmi * cholesterol_level', 'cholesterol_level * survived',
       'TIME_SINCE(beginning_of_treatment_date)', 'TIME_SINCE(diagnosis_date)',
       'YEAR(beginning_of_treatment_date)', 'YEAR(diagnosis_date)',
       'YEAR(end_treatment_date)'],
      dtype='object')


In [34]:
from sklearn.feature_selection import mutual_info_classif

# Separate features and target
X = df_cleaned.drop(columns=['survived'])
y = df_cleaned['survived']

# Compute mutual information
mi = mutual_info_classif(X, y, random_state=42)

# Create a DataFrame of feature importances
mi_df = pd.DataFrame({'feature': X.columns, 'mutual_info': mi})

# Sort by mutual information
mi_df = mi_df.sort_values(by='mutual_info', ascending=False)

# Display top 10 important features
print("Top 10 Important Features by Mutual Information:")
print(mi_df.head(10))


Top 10 Important Features by Mutual Information:
                          feature  mutual_info
154          PERCENTILE(survived)     0.603801
57             ABSOLUTE(survived)     0.549268
125                age * survived     0.527143
107                survived / age     0.527132
140  cholesterol_level * survived     0.526726
109  survived / cholesterol_level     0.526711
108                survived / bmi     0.526650
136                bmi * survived     0.526640
182       other_cancer - survived     0.447882
84        hypertension + survived     0.396960


In [ ]:
# Categorical vs Numerical columns
categorical_cols = df.select_dtypes(include=['object', 'bool']).columns
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns

# # Plot distribution of numeric columns
# for col in numerical_cols:
#     plt.figure(figsize=(8, 4))
#     sns.histplot(df[col].dropna(), kde=True)
#     plt.title(f'Distribution of {col}')
#     plt.show()

# # Plot count plots for categorical columns
# for col in categorical_cols:
#     plt.figure(figsize=(8, 4))
#     sns.countplot(data=df, x=col)
#     plt.title(f'Count Plot of {col}')
#     plt.xticks(rotation=45)
#     plt.show()

# # Correlation heatmap for numerical columns
# plt.figure(figsize=(15, 10))
# corr = df[numerical_cols].corr()
# sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm')
# plt.title('Correlation Heatmap')
# plt.show()

# # Pairplot for a few important features
# important_features = ['age', 'bmi', 'cholesterol_level', 'hypertension', 'survived']
# sns.pairplot(df[important_features], hue='survived', diag_kind='kde')
# plt.show()

# # Boxplot for outlier detection
# for col in numerical_cols:
#     plt.figure(figsize=(8, 4))
#     sns.boxplot(x=df[col])
#     plt.title(f'Boxplot for {col}')
#     plt.show()

# # Analysis of categorical columns with survived target
# for col in categorical_cols:
#     plt.figure(figsize=(8, 4))
#     sns.countplot(data=df, x=col, hue='survived')
#     plt.title(f'{col} vs Survived')
#     plt.xticks(rotation=45)
#     plt.show()

# # Distribution of 'survived' target variable
# plt.figure(figsize=(6, 4))
# sns.countplot(x=df['survived'])
# plt.title('Survival Distribution')
# plt.show()

# # Time-related analysis: converting any date columns to datetime
# date_cols = [col for col in df.columns if 'date' in col.lower()]
# for col in date_cols:
#     df[col] = pd.to_datetime(df[col], errors='coerce')

# # Example: Time since diagnosis vs survival
# if 'diagnosis_date' in df.columns:
#     df['year_of_diagnosis'] = df['diagnosis_date'].dt.year
#     plt.figure(figsize=(8, 6))
#     sns.histplot(data=df, x='year_of_diagnosis', hue='survived', multiple='stack')
#     plt.title('Year of Diagnosis vs Survival')
#     plt.show()
